# get Cause - efffect phrase from the sentence.

In [1]:
#filtered_rows.reset_index(drop=True, inplace=True)


## 1. load cause-effect data

In [2]:
import ast
import pandas as pd
#'data/in/semeval/semeval_causalityDetection.csv'
#'data/in/SCITE/SCITE_TrueFalse.csv'
#'data/in/RAG/ADE/ADE_causalityTEST.csv'
#load semeval data with embed

#'data/in/semeval/sem_CE_test.csv'
#'data/in/RAG/ADE/ADE_CE_extract_TEST.csv')
#'data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUT.csv'


pm_DB = pd.read_csv('data/in/KNN_RAG/SCITE_CE_extract_INPUT.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]
instances = pm_DB['sentence'].to_list()
len(instances)

191

## 2. load few shot database

In [3]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()
len(examples)

2365

## 3. identify cause- effect phrase

In [4]:
#load causality input sentence.
#input_sentence_list = filtered_rows['sentence'].to_list()
#gpt to extract cause effect and a causality phrase.
#: few shot: example1: , causality phrase: .

In [5]:
from openai import OpenAI
client = OpenAI(api_key='enter your API key here..')

#+ fewshot_examples
def C_E_CP_Identifier(input_sentence, fewshot_examples):
    
    response = client.chat.completions.create(
      model="gpt-4o",
      response_format= {"type": "json_object"},
      messages=[
        {
          "role": "system",
          "content": '''
  
          You will be given a sentence that shows a causality relation. 
          Your task is to identify all the cause phrase, effect phrase, and causality phrase from
          that sentence. Below is the hints and rules for the task.

          Hints: In sentences showing a causality relation, you can identify a pair of phrases
          where one phrase causes the other (effect). There will also be a phrase between the
          cause and effect phrases that indicates the causality relation; this is the 
          causality phrase. And in most sentences causality phrase can be seen between the 
          cause and effect phrase.
           

          Your output should be in JSON format with keys cause, effect, and causality_phrase:
          {"cause1": "", "effect1": "", "causality_phrase1": "", "cause2": "", "effect2": "", "causality_phrase2": "",
          "cause3": "", "effect3": "", "causality_phrase3": "", ....}. Make sure your JSON ouput is closed with } at the end.
          
          Rule 1: If you are not able to find the cause phrase, effect phrase, and causality phrase 
          from the given sentence you can output empty string for the keys in the json format 
          and do not guess the answer. 
          
          Rule 2: Ensure that there are no overlapping phrases between the cause phrase, effect phrase,
          and causality phrase. For instance, consider the input sentence "Fever is caused by virus." 
          Your identified output without overlap should be:
          {"cause": "virus", "effect": "fever", "causality_phrase": "caused by"}
          Your output should not contain overlap, as shown below:
          {"cause": "by virus", "effect": "fever is caused", "causality_phrase": "caused by"}
          This means that each phrase (cause, effect, and causality phrase) should be distinct 
          and not overlap with each other.
          
          Rule 3: There could be multiple cause-effect phrases in the sentence. You should extract all 
          of them. For instance, consider the input sentence "Mutations in MCFD2 causing F5F8D are null
          mutations but also missense mutations, which cause destabilization of its flexible structure
          and the loss of its interaction with ERGIC-53 .". 
          Your identified output will be:
          {"cause1": "Mutations in MCFD2", "effect1": "F5F8D", "causality_phrase1": "causing", 
          "cause2": "missense mutations", "effect2": "destabilization of its flexible structure", "causality_phrase2": "cause",
          "cause3": "missense mutations", "effect3": "the loss of its interaction with ERGIC-53", "causality_phrase3": "cause"

          Below are 10 example sentences that show causality relations, with their cause and
          effect phrases delimited between the XML tags <cause>, </cause>, <effect>, and 
          </effect>. The corresponding causality phrases are also provided.
          

                   
          ''' + fewshot_examples
            
        },
        {
          "role": "user",
          "content": input_sentence 
        }
      ],
      temperature=0.0,
      seed = 1,
      max_tokens=200,
      top_p=1
    )
    
    return response.choices[0].message.content


In [6]:
import json
import re

def close_json_string(json_string):
    
    
    # Remove entries where "cause", "effect", and "causality_phrase" are all empty
    pattern = re.compile(r'"cause\d+":\s*"",\s*"effect\d+":\s*"",\s*"causality_phrase\d+":\s*""\s*,?\s*')
    json_string = re.sub(pattern, '', json_string)
    
    # Remove incomplete index entries
    incomplete_index_pattern = re.compile(r'"cause\d+":\s*""\s*,?\s*')
    json_string = re.sub(incomplete_index_pattern, '', json_string)
    
    # Remove trailing comma if exists
    json_string = re.sub(r',\s*}$', '}', json_string.strip())
    json_string = re.sub(r',\s*$', '', json_string.strip())
    
    
    
    # Find the last complete index
    last_index = re.findall(r'"cause\d+":\s*"[^"]*",\s*"effect\d+":\s*"[^"]*",\s*"causality_phrase\d+":\s*"[^"]*"', json_string)
    
    # If found, truncate the JSON string after the last complete index
    if last_index:
        last_index_end_pos = json_string.rfind(last_index[-1]) + len(last_index[-1])
        json_string = json_string[:last_index_end_pos]
    
    # Count the number of opening and closing braces
    open_braces = json_string.count('{')
    close_braces = json_string.count('}')
    
    # Add missing closing braces if needed
    if open_braces > close_braces:
        json_string += '}' * (open_braces - close_braces)
    
    return json_string

In [7]:
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
filtered_rows = pm_DB

sent_cause_effect_list = []
causality_phrase_list = []

fewshotfound_inputsentence_index=[]
for i in range(len(filtered_rows)):
    input_sentence = filtered_rows['sentence'][i]
    #input_sentence = 'fever is caused by virus'
    
    #print('input_sentence', input_sentence)
    
    # few shot examples augmentation.
    # few shot examples augmentation.
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_examples = [examples[j] for j in most_similar_indices]

    fewshot_examples = {f'example {k + 1}': sentence for k, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)   
    #print('fewshot_examples', fewshot_examples)

    #fewshot_examples = get_fewshotexamples_C_E_CP(fewshot_index) 
    
    #print('fewshotfound_inputsentence_index', fewshotfound_inputsentence_index)
    
    # output generation
    out = C_E_CP_Identifier(input_sentence, fewshot_examples)
    out = close_json_string(out)
    print('out : ', out)

    
    # Attempt to parse the JSON string into a dictionary
    try:
        data = json.loads(out)
    except json.JSONDecodeError as e:
        print("Failed to parse JSON:", e)
        sent_cause_effect_list.append([])
        causality_phrase_list.append([])
        continue  # Skip to the next iteration if there is a JSON error

    # Parse JSON string into a dictionary
    data = json.loads(out)

    # Initialize lists
    sent_cause_effect = []
    causality_phrase_row = []

    # Iterate through the keys and collect values
    i_out = 1
    while f'cause{i_out}' in data and f'effect{i_out}' in data and f'causality_phrase{i_out}' in data:
        sent_cause_effect.append((input_sentence, data[f'cause{i_out}'], data[f'effect{i_out}']))
        causality_phrase_row.append(data[f'causality_phrase{i_out}'])
        i_out += 1

    # Print the results
    print("sent_cause_effect:", sent_cause_effect)
    #print("causality_phrase_row:", causality_phrase_row)
    sent_cause_effect_list.append(sent_cause_effect)
    causality_phrase_list.append(causality_phrase_row)
    print('i:', i)
    
    #if i==3:
        #break;
    

out :  {"cause1": "conventional electricity sources", "effect1": "pollution", "causality_phrase1": "reduce",
"cause2": "pollution", "effect2": "smog", "causality_phrase2": "causes",
"cause3": "pollution", "effect3": "acid rain", "causality_phrase3": "causes",
"cause4": "pollution", "effect4": "respiratory illness", "causality_phrase4": "causes",
"cause5": "pollution", "effect5": "global warming", "causality_phrase5": "causes"}
sent_cause_effect: [('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'conventional electricity sources', 'pollution'), ('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid rain ,  respiratory illness  and  global warming .', 'pollution', 'smog'), ('Using solar electricity instead of conventional electricity sources helps to reduce  pollution  that causes  smog ,  acid

out :  {"cause1": "steam", "effect1": "backpressure on the VTD", "causality_phrase1": "caused", "cause2": "backpressure on the VTD", "effect2": "the rupture disc opening", "causality_phrase2": "resulting in"}
sent_cause_effect: [(' The steam  caused  a backpressure  on the VTD, resulting in  the rupture disc opening .', 'steam', 'backpressure on the VTD'), (' The steam  caused  a backpressure  on the VTD, resulting in  the rupture disc opening .', 'backpressure on the VTD', 'the rupture disc opening')]
i: 20
out :  {"cause": "rapid power acceleration", "effect": "pressure", "causality_phrase": "was caused by"}
sent_cause_effect: []
i: 21
out :  {"cause1": "the sudden displacement of large volumes of water", "effect1": "tsunamis", "causality_phrase1": "are caused by"}
sent_cause_effect: [(' Tsunamis  are caused by  the sudden displacement  of large volumes of water.', 'the sudden displacement of large volumes of water', 'tsunamis')]
i: 22
out :  {"cause1": "the injury", "effect1": "disc

out :  {"cause1": "collision between a ship and a barge", "effect1": "oil spill", "causality_phrase1": "caused by"}
sent_cause_effect: [(" An oil spill  caused by  a collision  between a ship and a barge closed the ship channel servicing the nation's second largest port.", 'collision between a ship and a barge', 'oil spill')]
i: 45
out :  {"cause1": "the storms", "effect1": "the severe devastations", "causality_phrase1": "caused by"}
sent_cause_effect: [('The funds raised through this appeal have a direct impact on the people affected by  the severe devastations  caused by  the storms .', 'the storms', 'the severe devastations')]
i: 46
out :  {"cause": "the purchase", "effect": "disappointment", "causality_phrase": "from"}
sent_cause_effect: []
i: 47
out :  {"cause1": "drugs he sold", "effect1": "overdose death of Matthew Lessard, 19, of Lowell", "causality_phrase1": "had caused"}
sent_cause_effect: [(' The drugs  he sold had caused  the overdose death  of Matthew Lessard, 19, of Lowel

out :  {"cause1": "crashes", "effect1": "collapse of much of the World Trade Center complex", "causality_phrase1": "resulted in", "cause2": "crashes", "effect2": "destruction of part of the southwest side of the Pentagon", "causality_phrase2": "resulted in"}
sent_cause_effect: [(' The crashes  resulted in  the collapse  of much of the World Trade Center complex,  the destruction  of part of the southwest side of the Pentagon.', 'crashes', 'collapse of much of the World Trade Center complex'), (' The crashes  resulted in  the collapse  of much of the World Trade Center complex,  the destruction  of part of the southwest side of the Pentagon.', 'crashes', 'destruction of part of the southwest side of the Pentagon')]
i: 77
out :  {"cause": "germs", "effect": "flu, colds or sore throat", "causality_phrase": "that were causing"}
sent_cause_effect: []
i: 78
out :  {"cause": "a participant complaint", "effect": "an investigative lead", "causality_phrase": "is the source of"}
sent_cause_effect

out :  {"cause": "pollution caused by the buring of coal", "effect": "light-colored tree trunks became darker", "causality_phrase": "due to"}
sent_cause_effect: []
i: 102
out :  {"cause": "A spaghetti pie served at a church dinner", "effect": "the poisoning of one hundred people", "causality_phrase": "had caused"}
sent_cause_effect: []
i: 103
out :  {"cause1": "chronic inflammation", "effect1": "cysts", "causality_phrase1": "are caused by"}
sent_cause_effect: [(' The cysts  are caused by  chronic inflammation  of the perichondrium with production of serous fluid between the perichondrium and cartilage.', 'chronic inflammation', 'cysts')]
i: 104
out :  {"cause1": "sinus infections", "effect1": "terrible headaches", "causality_phrase1": "from", "cause2": "sinus infections", "effect2": "taking antibiotics", "causality_phrase2": "resulted in"}
sent_cause_effect: [('I used to get  terrible headaches  from  sinus infections  that resulted in  taking antibiotics  a few times a year.', 'sinus 

out :  {"cause": "software", "effect": "drain", "causality_phrase": "caused"}
sent_cause_effect: []
i: 121
out :  {"cause": "widgets", "effect": "problem", "causality_phrase": "resembling HTC's own Sense UI widgets"}
sent_cause_effect: []
i: 122
out :  {"cause1": "steroids", "effect1": "diabetes", "causality_phrase1": "cause", "cause2": "steroids", "effect2": "hypertension", "causality_phrase2": "cause"}
sent_cause_effect: [('Liver plays key role in how  steroids  cause  diabetes  and  hypertension .', 'steroids', 'diabetes'), ('Liver plays key role in how  steroids  cause  diabetes  and  hypertension .', 'steroids', 'hypertension')]
i: 123
out :  {"cause1": "Zinc", "effect1": "growth", "causality_phrase1": "is essential for", "cause2": "Zinc", "effect2": "cell division", "causality_phrase2": "is essential for"}
sent_cause_effect: [(' Zinc  is essential for  growth  and  cell division .', 'Zinc', 'growth'), (' Zinc  is essential for  growth  and  cell division .', 'Zinc', 'cell divisio

out :  {"cause1": "a hormone called progesterone", "effect1": "changes now seen in the endometrium", "causality_phrase1": "caused by"}
sent_cause_effect: [(' The changes  now seen in the endometrium are caused by  a hormone  called progesterone.', 'a hormone called progesterone', 'changes now seen in the endometrium')]
i: 149
out :  {"cause1": "a cholera outbreak", "effect1": "the deaths", "causality_phrase1": "are from"}
sent_cause_effect: [('About 30 ducks were found dead in Klamath County in the past two days, and officials are investigating whether  the deaths  are from  a cholera outbreak .', 'a cholera outbreak', 'the deaths')]
i: 150
out :  {"cause": "a virus", "effect": "Plantar warts", "causality_phrase": "are caused by"}
sent_cause_effect: []
i: 151
out :  {"cause": "switching process", "effect": "interference", "causality_phrase": "triggered by"}
sent_cause_effect: []
i: 152
out :  {"cause": "These chemical studies", "effect": "proof of structure of the indole components of 

out :  {"cause1": "Givers", "effect1": "moral strength", "causality_phrase1": "gain", "cause2": "Givers", "effect2": "happiness", "causality_phrase2": "gain"}
sent_cause_effect: [('Givers gain  moral strength  and  happiness  from  giving .', 'Givers', 'moral strength'), ('Givers gain  moral strength  and  happiness  from  giving .', 'Givers', 'happiness')]
i: 174
out :  {"cause1": "a strong offshore earthquake", "effect1": "a tsunami", "causality_phrase1": "is generated by"}
sent_cause_effect: [('When  a tsunami  is generated by  a strong offshore earthquake , its first waves would reach the outer coast minutes after the ground stops shaking.', 'a strong offshore earthquake', 'a tsunami')]
i: 175
out :  {"cause": "keratinophilic fungi", "effect": "cutaneous mycoses", "causality_phrase": "are caused by"}
sent_cause_effect: []
i: 176
out :  {"cause1": "the Prestige oil spill", "effect1": "the ecological catastrophe", "causality_phrase1": "caused by", "cause2": "the Exxon Valdez", "effec

In [8]:
len(filtered_rows)

191

In [9]:
#filtered_rows = filtered_rows.drop('GPT_causality_label', axis = 1)
#filtered_rows = filtered_rows.drop('patternmatcher_label', axis = 1)
len(fewshotfound_inputsentence_index)

0

In [10]:
filtered_rows['sent_cause_effect'] = sent_cause_effect_list
filtered_rows['causality_phrase'] = causality_phrase_list
filtered_rows

,Unnamed: 0.1,Unnamed: 0,sentence,embedings,sent_cause_effect,causality_phrase
0,0,0,Using solar electricity instead of conventiona...,"[0.030286945402622223, 0.00630235718563199, -0...",[(Using solar electricity instead of conventio...,"[reduce, causes, causes, causes, causes]"
1,1,1,"Thus, evaluating capital punishment as a form ...","[-0.011410091072320938, 0.005278998054563999, ...","[(Thus, evaluating capital punishment as a for...",[that result in]
2,2,2,Sudden death from inhalation of petroleum ...,"[-0.0007025353843346238, 0.00263791810721159, ...",[],[]
3,3,3,Stimulation of the posterior horn of the med...,"[-0.03430342674255371, 0.01450358610600233, 0....",[],[]
4,4,4,Sip the tea slowly to reduce stomach pain fr...,"[0.005228457972407341, -0.011144616641104221, ...",[(Sip the tea slowly to reduce stomach pain ...,[from]
...,...,...,...,...,...,...
186,186,186,The disruption has been caused by work bei...,"[-0.019408993422985077, -0.003347919089719653,...",[],[]
187,187,187,"Most illnesses, including colds and flu , c...","[0.007080608978867531, -0.004280312918126583, ...","[(Most illnesses, including colds and flu ,...","[cause, cause]"
188,188,188,Every child and young person seen at Starship ...,"[0.019172143191099167, 0.013259614817798138, 0...",[(Every child and young person seen at Starshi...,[from]
189,189,189,The movie gives the inaccurate impression ...,"[-0.04086083546280861, -0.011357069946825504, ...",[],[]


In [11]:
#SCITE

pm_DB = pd.read_csv('data/in/RAG/c_e_Extraction/SCITE_CE_extract_INPUTkeys.csv')
pm_DB['keys'] = pm_DB['keys'].apply(eval)



C_t = pm_DB['keys'].to_list()


C_p = sent_cause_effect_list

In [12]:
len(C_t)

296

In [13]:
C_p = [tup for sublist in C_p for tup in sublist]
len(C_p[1])


3

In [14]:
def get_score(true_phrase, pred_phrase ):
    scores=[]
    p = true_phrase
    window_size = len(p)

        # Slide through input_sentence with a window size of 8
    if (pred_phrase) =="":
        print('no pred_phrase')
        return 0
    else: 
        for i in range(len(pred_phrase) - window_size + 1):
            window = pred_phrase[i:i + window_size]
            score = fuzz.ratio(window, p)
            scores.append(score)
        if len(scores) ==0:
            print('prediction length lower', pred_phrase, 'p:', p)
            return 0
        else:
            return max(scores)

In [15]:
len(C_p)

188

In [16]:
from fuzzywuzzy import fuzz
prediction= []
pred_label = []
for i in range(len(C_p)):
    #print(i)
    labeled = 0   
    if (C_p[i] != []):
        if len(C_p[i]) == 3:           
            
            sent = C_p[i][0]
            ####print('sent: ',sent )
            cause_pred = C_p[i][1]
            effect_pred = C_p[i][2]
            
            for i_t in range(len(C_t)):
                #get index of all tuples which match the sent and create sent_found = []
                # List to store indices
                sent_found = []

                # Check if sent is in the C_t at index 0
                for index, (sentence, _, _) in enumerate(C_t):
                    if sentence == sent:
                        sent_found.append(index)
            ####print('sent_found: ', sent_found)
            
            
            #iterate through sent_found and pass each cause_t and effect_t to get score: 
            for ind in sent_found:
                cause_t = C_t[ind][1]
                effect_t = C_t[ind][2]
                pred_c=0
                pred_e=0
                ##print('cause_t: ', cause_t)
                ##print('effect_t: ', effect_t)
                ##print('cause_pred: ', cause_pred)
                ##print('effect_pred: ', effect_pred)
                
                score = get_score(cause_t.lower(), cause_pred.lower())################ non -medcause
                    #score = fuzz.ratio(cause_pred, cause_t)################medcause
                if score >90:################90 non -medcause
                    pred_c = 1
                else:
                    pred_c=0
                    


                score = get_score(effect_t.lower(), effect_pred.lower())################non - medcause
                    #score = fuzz.ratio(effect_pred, effect_t)################medcause

                if score >90: ################90 non -medcause
                    pred_e = 1
                else:
                    pred_e=0



                if ((pred_e ==1) & (pred_c ==  1)):
                    prediction.append(1)
                    
                    labeled = 1
                    #pred_label.append(labeled)
                    #print('out:', 1)
    
    
    
    
    pred_label.append(labeled)
                    
    print(labeled, i)



                
    

 

prediction length lower pollution p: respiratory illness
prediction length lower pollution p: global warming
0 0
prediction length lower smog p: acid rain
prediction length lower smog p: respiratory illness
prediction length lower smog p: global warming
1 1
prediction length lower acid rain p: respiratory illness
prediction length lower acid rain p: global warming
1 2
1 3
prediction length lower global warming p: respiratory illness
1 4
1 5
1 6
1 7
0 8
prediction length lower hull p: the hull
0 9
1 10
1 11
1 12
1 13
1 14
1 15
prediction length lower suffering p: the suffering
prediction length lower exploitation p: social injustice
prediction length lower suffering p: the suffering
prediction length lower suffering p: the suffering
prediction length lower suffering p: the suffering
0 16
prediction length lower suffering p: the suffering
prediction length lower suffering p: the suffering
prediction length lower suffering p: the suffering
prediction length lower suffering p: the sufferin

1 144
0 145
prediction length lower virus p: the virus
0 146
prediction length lower virus p: the virus
0 147
0 148
1 149
prediction length lower vascular dilatation p: the vascular dilatation
prediction length lower vascular dilatation p: the vascular dilatation
0 150
prediction length lower vascular dilatation p: the vascular dilatation
prediction length lower vascular dilatation p: the vascular dilatation
0 151
0 152
prediction length lower suffering p: the suffering
0 153
prediction length lower damage p: the damage
0 154
prediction length lower tissues p: the damage
0 155
0 156
prediction length lower stress p: the stress
prediction length lower bleeding stomach ulcer p: a bleeding stomach ulcer
prediction length lower stress p: hard work
prediction length lower bleeding stomach ulcer p: a bleeding stomach ulcer
0 157
prediction length lower hard work p: the stress
prediction length lower bleeding stomach ulcer p: a bleeding stomach ulcer
prediction length lower bleeding stomach u

In [17]:
prec=sum(prediction)/len(C_p)
rec = sum(prediction)/len(C_t)
F1 = 2 *(prec * rec)/(prec + rec)
print('F1: ', F1)
print('prec: ', prec)
print('rec: ', rec)


F1:  0.45867768595041325
prec:  0.5904255319148937
rec:  0.375


In [18]:
len(pred_label)

188

In [19]:
len(C_p)

188

In [20]:
SCITE_df = pd.DataFrame({'pred_label': pred_label, 'C_p' : C_p})
SCITE_df

,pred_label,C_p
0,0,(Using solar electricity instead of convention...
1,1,(Using solar electricity instead of convention...
2,1,(Using solar electricity instead of convention...
3,1,(Using solar electricity instead of convention...
4,1,(Using solar electricity instead of convention...
...,...,...
183,1,( Snails and slugs cause damage to seedli...
184,1,"(Most illnesses, including colds and flu , ..."
185,1,"(Most illnesses, including colds and flu , ..."
186,1,(Every child and young person seen at Starship...


In [21]:
#filtered_rows.to_csv('data/out/RAG_C_E_CP_Alzeimers.csv')#data/out/RAG_causality/semeval/semeval_
#filtered_rows.to_csv('data/out/RAG_causality/semeval/fewshot_sem_CE_detection.csv')#data/out/RAG_causality/semeval/semeval_

#fewshot_semeval_CE_detection
SCITE_df.to_csv('data/out/RAG_causality/task2/GPT4/KNN_SCITE_CE_extract.csv')
#filtered_rows.to_csv('data/out/RAG_causality/task2/GPT4/zero_sem_CE_extract.csv')#zero,rand,RAG|sem,ADE,SCITE


In [22]:
#filtered_rows= filtered_rows[filtered_rows['predictions'] == 0]
#filtered_rows.to_csv('data/out/RAG_causality/semeval/zero_sem_RAG_C_E_pred0.csv')#data/out/RAG_causality/semeval/semeval_


In [23]:
prec= 0.73
rec = 0.73
2*(prec*rec)/(prec+rec)

0.7299999999999999